In [1]:
using LinearAlgebra
using PolyChaos
using QuadGK
using Plots
using Base.Threads

println(Threads.nthreads())

sys_size = 2

function spectral_func(input::AbstractString, D::Float64, g::Float64)
    inband = x -> (-D <= x <= D)
        
    if input == "flat"
        J = x -> inband(x) ? 1/(2D) : 0.0
    elseif input == "elliptical"
        J = x -> inband(x) ? sqrt(1 - (x/D)^2) : 0.0
    elseif input == "ohmic"
        J = x -> inband(x) ? abs(x) : 0.0
    elseif input == "lorentzian"
        J = x -> 1/(1 + (x/D)^2)
    else
        error("spectral function type not recognized")
    end
    # normalization
    norm = quadgk(J, -D, D)[1]           
    Jnorm = x -> g*D/pi * J(x) / norm
    return Jnorm
end

function thermofield_transform(J, beta::Float64, mu::Float64) #spectral function, inverse temp, chemical potential
    """thermofield purification using fermi function ancilla"""
    fermi(k) = 1/(1 + exp(beta*k - beta*mu))
    J1 = w -> J(w) * fermi(w) #filled mode spectral density
    J2 = w -> J(w) * (1 - fermi(w)) #empty mode spectral density
    return J1, J2
end

function chain_map(J, N::Int64, D::Float64)
    """calculates family of monic orthogonal polynomials w.r.t the measure J(x) up to the Nth term.
    returns the coefficients alpha and beta from the recurrence relation of the family."""
    supp = (-D, D)
    meas = Measure("bath", J, supp, false, Dict())
    ortho_poly = OrthoPoly("bath_op", N, meas; Nquad=10000)   
    chain = coeffs(ortho_poly)                                  
    Es = chain[1:N,1] #site energies
    ts = sqrt.(chain[1:N,2]) #site hoppings (first term is system hopping)
    return Es, ts
end

function prepare_corrs(N, sys_occ)
    nd0 = sys_occ                 
    
    I_L  = I(N)                
    Z_L  = zeros(N, N)

    # assemble C0
    C0 = zeros(ComplexF64, 2N+1, 2N+1)
    C0[N+1,N+1] = nd0 + 0im
    C0[1:N, 1:N] .= I_L    
    C0[N+2:end, N+2:end] .= Z_L
    C0 = Hermitian(C0)
    return C0
end


function HamiltonianCorrs(N::Int64, Es::Float64, E1, t1, E2, t2)
"""exact Hamiltonian for chain mapped OQS"""
    E1 = reverse(E1) #empty chain onsite energies
    h1 = reverse(t1) #empty chain NN couplings
    d = Vector{Float64}(undef, 2*N+1)     # diagonal 
    e = Vector{Float64}(undef, 2*N)       # off-diagonal
    d[1:N] .= E1
    d[N+1] = Es
    d[N+2:2N+1] .= E2
    e[1:N] .= t1
    e[N+1:2N] .= t2
    H = SymTridiagonal(d, e)            # Hermitian and tridiagonal
    return H
end

function evolve_corrs(C0, H, dt, tmax)
    Cs = Vector{Array{ComplexF64}}(undef, 0)
    times = collect(0:dt:tmax)
    C0 = Matrix(C0)
    H = Matrix(H)

    for t in times
        U = exp(-im*t*H)
        C = U * C0 * U'
        push!(Cs, C)
    end
    return Cs
end

8


evolve_corrs (generic function with 1 method)

In [ ]:
function trace_dist(rho1, rho2)
    return 0.5 * abs(rho1 - rho2)
end

function HS_dist(A, B, n, N)
    
    Ma = A[N+1-n:N+1+n,N+1-n:N+1+n]
    Mb = B[N+1-n:N+1+n,N+1-n:N+1+n]
    Id = I(2n+1)
    d2 = sqrt(0.5 * (det(Id - 2*Ma + 2*Ma*Ma) + det(Id - 2*Mb + 2*Mb*Mb) - 2*det(Id - Ma - Mb + 2*Ma*Mb)))
    return d2 #each term in D is the 'overlap' of the two correlation matrices
end

function block_entropy(C, n, N)
    Csub = C[N+1-n:N+1+n,N+1-n:N+1+n]
    Id = Matrix(1.0I, 2n+1, 2n+1)
    evals1 = eigvals(Csub)
    evals2 = eigvals(Id .- Csub)
    S = -sum(evals1 .* log.(evals1) .+ evals2 .* log.(evals2))
    return S
end

function VNentropy(C)
    Id = Matrix(1.0I, size(C))
    evals1 = eigvals(C)
    evals2 = eigvals(Id .- C)
    S = -sum(evals1 .* log.(evals1) .+ evals2 .* log.(evals2))
    return S
end

function mutual_info(C, N)
    """mutual info beetween left and right chain"""
    Ctr = C[1:N, N+2:end]
    Cbl = C[N+2:end, 1:N]
    Cfull = C[1:N, 1:N]
    Cempty = C[N+2:end, N+2:end]

    Csub = hvcat((2,2), Cfull, Ctr, Cbl, Cempty)

    return real(VNentropy(Cfull) + VNentropy(Cempty) - VNentropy(Csub))
end

function BLP(TD)
    sigma = TD[2:end] .- TD[1:end-1]
    meas = sum(sigma[sigma .>= 0.0])
    return meas
end

BLP (generic function with 1 method)

#simulation params
N = 50
dt = 0.1
tmax = 100.0
times = collect(0:dt:tmax)

Es = 0.0

for D in collect(0.5:0.1:1.0)
    for g in collect(0.1:0.1:0.5)
        J = spectral_func("elliptical", D, g)
        for beta in vcat(collect(0.0:1.0:10.0),[100.0,200.0,300.0])
            for mu in [-0.2,0.0,0.2]
                J1, J2 = thermofield_transform(J, beta, mu)
                E1, t1 = chain_map(J1, N, D)
                E2, t2 = chain_map(J2, N, D)
                H = HamiltonianCorrs(N, Es, E1, t1, E2, t2)

                C0f = prepare_corrs(N, 1.0) #filled initial
                C0e = prepare_corrs(N, 0.0) #empty initial
                Csf = evolve_corrs(C0f, H, dt, tmax) #filled evolution
                Cse = evolve_corrs(C0e, H, dt, tmax) #empty evolution
                MIf = zeros(length(times))
                MIe = zeros(length(times))
                for i in 1:length(times)
                    Cf = Csf[i] 
                    Ce = Cse[i] 
                    MIf[i] = mutual_info(Cf, N)
                    MIe[i] = mutual_info(Ce, N)
                end
                p = plot(times, MIf, label="Filled initial state", title="\$\\beta=$beta,g=$g,D=$D,\\mu=$mu\$",
                        xlabel="Time", ylabel="Mutual Info between chains", lw=2, c=:blue, dpi=500)
                plot!(p, times, MIe, label="Empty initial state", lw=2, c=:red)
                savefig(p,"C:\\Users\\ben_b\\Documents\\Git Repos\\AgeingManyBodyQuantumSys\\2ndTerm\\images\\MutualInfo\\MutInfo_beta$(beta)g$(g)D$(D)mu$(mu).png",)
            end
        end
    end
end

In [ ]:
N=50
dt = 0.1
tmax = 100.0
times = collect(0:dt:tmax)

Es=0.0
D=1.0
mu=0.0
betas = collect(0.0:10.0:100.0)
gs = collect(0.05:0.05:2.0)

data = zeros(length(betas), length(gs))

for (i,beta) in enumerate(betas)
    @threads for j in eachindex(gs)
        g = gs[j]
        J = spectral_func("elliptical", D, g)
        J1, J2 = thermofield_transform(J, beta, mu)

        E1, h1 = chain_map(J1, N, D)
        E2, h2 = chain_map(J2, N, D)

        H = HamiltonianCorrs(N, Es, E1, E2, h1, h2)
        C0_full = prepare_corrs(N,1.0)
        C0_empty = prepare_corrs(N,0.0)
        Cs_full = evolve_corrs(C0_full, H, dt, tmax)
        Cs_empty = evolve_corrs(C0_empty, H, dt, tmax)

        TD = zeros(length(times))
        for t in 1:length(times)
            rho1 = Cs_full[t][N+1,N+1]
            rho2 = Cs_empty[t][N+1, N+1]
            TD[t] = trace_dist(rho1, rho2)
        end
        data[i,j] = BLP(TD)

        println("$beta")
    end    
end



In [ ]:
h = heatmap(gs, betas, data, xlabel="g", ylabel="\$1/K_B T\$", title="BLP Measure, \$D=$D,\\mu=$mu\$")

display(h)